In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
#import cv2
import numpy as np
import os

l = tf.keras.layers
imageHeight = 512
imageWidth = 512

sigma = 0.06
focal_alpha = 2
focal_beta = 4
lambda_size = 0.1
lambda_offset = 1.0

filterDim = [256, 384, 384, 384, 512]

train_files_names = os.listdir('/home/roy/data/coco/train_tf/')
train_files = ['/home/roy/data/coco/train_tf/'+item for item in train_files_names]

resize_width = 550
resize_height = 550
image_width = 512
image_height = 512
image_width_delta = resize_width - image_width
image_height_delta = resize_height - image_height 
batch_size = 64
valid_batch_size = 1
epoch_size = 26332
category_num = 80
jitter = 0.3
vector_size = 1+4+category_num
label_vector_size = 1+4+category_num
labels = ['umbrella',
 'sandwich',
 'handbag',
 'person',
 'snowboard',
 'cell phone',
 'traffic light',
 'potted plant',
 'toaster',
 'baseball glove',
 'cow',
 'surfboard',
 'remote',
 'toilet',
 'baseball bat',
 'giraffe',
 'book',
 'bottle',
 'stop sign',
 'frisbee',
 'boat',
 'sheep',
 'mouse',
 'motorcycle',
 'car',
 'bird',
 'pizza',
 'bed',
 'kite',
 'zebra',
 'broccoli',
 'cat',
 'chair',
 'bench',
 'teddy bear',
 'tennis racket',
 'laptop',
 'sink',
 'sports ball',
 'skateboard',
 'parking meter',
 'carrot',
 'hair drier',
 'banana',
 'wine glass',
 'scissors',
 'spoon',
 'cake',
 'fire hydrant',
 'dog',
 'backpack',
 'airplane',
 'clock',
 'keyboard',
 'truck',
 'bicycle',
 'skis',
 'bus',
 'hot dog',
 'dining table',
 'cup',
 'toothbrush',
 'horse',
 'elephant',
 'refrigerator',
 'knife',
 'suitcase',
 'apple',
 'donut',
 'couch',
 'train',
 'microwave',
 'bear',
 'oven',
 'bowl',
 'orange',
 'tv',
 'tie',
 'vase',
 'fork']

/home/roy/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/roy/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/roy/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/roy/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
def _label_fn(bbox):
    stride_height = image_height//4
    stride_width = image_width//4
    labels = np.zeros((stride_height, stride_width, label_vector_size)) 
    box_num, _ = bbox.shape
    index_x = np.tile(range(stride_width), (stride_height,1))
    index_y = np.transpose(index_x)
    for i in range(box_num):
        center_y = int(bbox[i,0]/4)
        center_x = int(bbox[i,1]/4)
        box_height = bbox[i,2]/4
        box_width = bbox[i,3]/4
        box_size = box_height*box_width
        category = bbox[i,4]
        truth = np.exp(-(np.square(index_x-center_x)+np.square(index_y-center_y))/(2*sigma*box_size))
        labels[:,:,category] = np.maximum(truth, labels[:,:,category])
        labels[center_y,center_x,-5] = 1.0
        labels[center_y,center_x,-4] = float(box_height)
        labels[center_y,center_x,-3] = float(box_width)
        labels[center_y,center_x,-2] = bbox[i,0]/4 - center_y
        labels[center_y,center_x,-1] = bbox[i,1]/4 - center_x
    return labels.astype(np.float32)

def _parse_function(example_proto):
    features = {"image": tf.FixedLenFeature([], tf.string, default_value=""),
                "height": tf.FixedLenFeature([1], tf.int64, default_value=[0]),
                "width": tf.FixedLenFeature([1], tf.int64, default_value=[0]),
                "channels": tf.FixedLenFeature([1], tf.int64, default_value=[3]),
                "colorspace": tf.FixedLenFeature([], tf.string, default_value=""),
                "img_format": tf.FixedLenFeature([], tf.string, default_value=""),
                "label": tf.VarLenFeature(tf.int64),
                "bbox_xmin": tf.VarLenFeature(tf.int64),
                "bbox_xmax": tf.VarLenFeature(tf.int64),
                "bbox_ymin": tf.VarLenFeature(tf.int64),
                "bbox_ymax": tf.VarLenFeature(tf.int64),
                "filename": tf.FixedLenFeature([], tf.string, default_value="")
               }
    parsed_features = tf.parse_single_example(example_proto, features)
    
    label = tf.expand_dims(parsed_features["label"].values, 0)
    label = tf.cast(label, tf.int32)
    height = tf.squeeze(tf.cast(parsed_features["height"], tf.int32))
    width = tf.squeeze(tf.cast(parsed_features["width"], tf.int32))
    channels = parsed_features["channels"]
    filename = tf.expand_dims(parsed_features["filename"],0)
    #Get the bbox
    xmin = tf.cast(tf.expand_dims(parsed_features["bbox_xmin"].values, 0), tf.int32)
    xmax = tf.cast(tf.expand_dims(parsed_features["bbox_xmax"].values, 0), tf.int32)
    ymin = tf.cast(tf.expand_dims(parsed_features["bbox_ymin"].values, 0), tf.int32)
    ymax = tf.cast(tf.expand_dims(parsed_features["bbox_ymax"].values, 0), tf.int32)

    boxes = tf.concat([xmin,ymin,xmax,ymax], axis=0)
    boxes = tf.transpose(boxes, [1, 0])
    #Decode the image
    image_raw = tf.image.decode_jpeg(parsed_features["image"], channels=3)
    image_decoded = tf.image.convert_image_dtype(image_raw, tf.float32)
    
    dw = jitter*tf.cast(width, tf.float32)
    dh = jitter*tf.cast(height, tf.float32)
    new_ar = tf.truediv(tf.add(tf.cast(width, tf.float32), tf.random.uniform([1], minval=tf.math.negative(dw), maxval=dw)), \
                        tf.add(tf.cast(height, tf.float32), tf.random.uniform([1], minval=tf.math.negative(dh), maxval=dh)))
    nh, nw = tf.cond(tf.less(new_ar[0],1), \
                     lambda:(image_height, tf.cast(tf.cast(image_height, tf.float32)*new_ar[0], tf.int32)), \
                     lambda:(tf.cast(tf.cast(image_width, tf.float32)/new_ar[0], tf.int32), image_width))
    dx = tf.cond(tf.equal(image_width, nw), \
                 lambda:tf.constant([0]), \
                 lambda:tf.random.uniform([1], minval=0, maxval=(image_width-nw), dtype=tf.int32))
    dy = tf.cond(tf.equal(image_height, nh), \
                 lambda:tf.constant([0]), \
                 lambda:tf.random.uniform([1], minval=0, maxval=(image_height-nh), dtype=tf.int32))
    #image_resize = tf.image.per_image_standardization(tf.image.resize(image_decoded, [nh, nw]))
    image_resize = tf.image.resize(image_decoded, [nh, nw])
    image_distort = tf.image.per_image_standardization(image_resize)
    #image_distort = distort_color(image_resize)
    image_padded = tf.image.pad_to_bounding_box(image_distort, dy[0], dx[0], image_height, image_width)

    #Adjust the boxes
    xmin_new = tf.cast(tf.truediv(nw, width) * tf.cast(xmin,tf.float64), tf.int32) + dx
    xmax_new = tf.cast(tf.truediv(nw, width) * tf.cast(xmax,tf.float64), tf.int32) + dx
    ymin_new = tf.cast(tf.truediv(nh, height) * tf.cast(ymin,tf.float64), tf.int32) + dy
    ymax_new = tf.cast(tf.truediv(nh, height) * tf.cast(ymax,tf.float64), tf.int32) + dy
    boxes_width = xmax_new-xmin_new
    boxes_height = ymax_new-ymin_new
    boxes_area = boxes_width*boxes_height
    
    # Random flip flag
    random_flip_flag = tf.random.uniform([1], minval=0, maxval=1, dtype=tf.float32)
    def flip_box():
        xmax_flip = image_width - xmin_new
        xmin_flip = image_width - xmax_new
        image_flip = tf.image.flip_left_right(image_padded)
        return xmin_flip, xmax_flip, image_flip
    def notflip():
        return xmin_new, xmax_new, image_padded
    xmin_flip, xmax_flip, image_flip = tf.cond(tf.less(random_flip_flag[0], 0.5), notflip, flip_box)
    center_x = xmin_flip + (xmax_flip-xmin_flip)//2
    center_y = ymin_new + (ymax_new-ymin_new)//2
    boxes_new = tf.concat([center_y,center_x,(ymax_new-ymin_new),(xmax_flip-xmin_flip),label], axis=0)
    boxes_new = tf.transpose(boxes_new, [1, 0])
    
    label_new = tf.py_func(_label_fn, [boxes_new], tf.float32)
    features = {'images': image_flip}
    return features, label_new

In [3]:
def train_input_fn():
    '''
    train_files = tf.data.Dataset.list_files("/home/roy/data/coco/train/train_10.tfrecord")
    dataset_train = train_files.interleave(tf.data.TFRecordDataset, cycle_length=4, num_parallel_calls=4)
    '''
    dataset_train = tf.data.TFRecordDataset(train_files)
    dataset_train = dataset_train.shuffle(buffer_size=epoch_size)
    dataset_train = dataset_train.repeat(100)
    dataset_train = dataset_train.map(_parse_function, num_parallel_calls=12)
    #dataset_train = dataset_train.padded_batch(batch_size, \
    #                    padded_shapes=([None,None, None], [None, None], [None, None, None]))
    dataset_train = dataset_train.batch(batch_size)
    dataset_train = dataset_train.prefetch(batch_size)
    return dataset_train
    '''
    iterator = tf.data.Iterator.from_structure(dataset_train.output_types, dataset_train.output_shapes)
    image_augment, bbox_result, label_result = iterator.get_next()
    train_init_op = iterator.make_initializer(dataset_train)
    '''

In [ ]:
label_test = label_result[:,:,:,:80]
labels_obj_mask = tf.equal(label_test,1.0)
labels_noobj_mask = tf.less(label_test,1.0)
labels_obj = tf.boolean_mask(label_test, labels_obj_mask)
labels_noobj = tf.boolean_mask(label_test, labels_noobj_mask)
#preds_obj_k = tf.boolean_mask(labels, labels_obj_mask)

In [ ]:
sess = tf.Session()
sess.run(train_init_op)

In [ ]:
image, bbox, label, label_1, label_2, mask1 = sess.run([image_augment, bbox_result, label_result, labels_obj, labels_noobj, labels_obj_mask])
#image, label = sess.run([image_augment, label_result])

In [ ]:
image_bbox = bbox[0]
for i in range(image_bbox.shape[0]):
    cv2.rectangle(image[0], (image_bbox[i][0],image_bbox[i][1]), (image_bbox[i][2],image_bbox[i][3]), (0,255,0), 2)
plt.imshow(image[0])

In [ ]:
sess.close()

Encpasulate the conv and residual functions

In [4]:
def _conv(inputs, filters, kernel_size, strides, padding, bias=False, normalize=True, activation='relu'):
    output = inputs
    padding_str = 'same'
    if padding>0:
        output = l.ZeroPadding2D(padding=padding)(output)
        padding_str = 'valid'
    output = l.Conv2D(filters, kernel_size, strides, padding_str, use_bias=bias, \
                 kernel_initializer='he_normal', \
                 kernel_regularizer=tf.keras.regularizers.l2(l=5e-4))(output)
    if normalize:
        output = l.BatchNormalization(axis=3)(output)
    if activation=='relu':
        output = l.ReLU()(output)
    if activation=='relu6':
        output = l.ReLU(max_value=6)(output)
    if activation=='leaky_relu':
        output = l.LeakyReLU(alpha=0.1)(output)
    if activation=='sigmoid':
        output = tf.keras.activations.sigmoid(output)
    return output

def _residual(inputs, filters, strides):
    shortcut = inputs
    num_channels = shortcut.get_shape().as_list()[-1]
    output = _conv(inputs, filters, 3, strides, 1)
    output = _conv(output, filters, 3, 1, 0, False, True, 'linear')
    if num_channels != filters or strides != 1:
        shortcut = _conv(shortcut, filters, 1, strides, 0, False, True, 'linear')
    output = l.Add()([output, shortcut])
    output = l.ReLU()(output)
    return output

Define the hourglass function

In [5]:
def _hourglass(inputs, filterDim):
    #Left part, left_features image dimension, 1,1/2,1/4,1/8,1/16,1/32
    left_features = [inputs]
    for index, dim in enumerate(filterDim):
        output = _residual(left_features[-1], dim, 2)
        output = _residual(output, dim, 1)
        left_features.append(output)
    #Middle part
    output = left_features[-1]
    for i in range(5):
        output = _residual(output, filterDim[-1], 1)
    #Right part
    for index in reversed(range(len(filterDim))):
        output = _residual(output, filterDim[index], 1)
        output = _residual(output, filterDim[max(index-1, 0)], 1)
        output = l.UpSampling2D()(output)
        left_feature = _residual(left_features[index], filterDim[max(index-1, 0)], 1)
        left_feature = _residual(left_feature, filterDim[max(index-1, 0)], 1)
        output = l.Add()([output, left_feature])
    output = _conv(output, 256, 3, 1, 1)
    outputs = [output]
    #Prediction
    pred_keypoints = _conv(output, 256, 3, 1, 0, True, False, 'relu') 
    pred_keypoints = _conv(pred_keypoints, 80, 1, 1, 0, True, False, 'sigmoid') 
    outputs.append(pred_keypoints)
    pred_offset = _conv(output, 256, 3, 1, 0, True, False, 'relu') 
    pred_offset = _conv(pred_offset, 2, 1, 1, 0, True, False, 'linear') 
    outputs.append(pred_offset)
    pred_size = _conv(output, 256, 3, 1, 0, True, False, 'relu') 
    pred_size = _conv(pred_size, 2, 1, 1, 0, True, False, 'linear') 
    outputs.append(pred_size)
    return outputs

Define the base hourglass network

In [6]:
def HourglassNetwork():
    image = tf.keras.Input(shape=(imageHeight,imageWidth,3))   #512×512×3
    net = _conv(image, 128, 7, 2, 3)    #256×256×128
    net = _residual(net, 256, 2)        #128×128×256
    outputs_1 = _hourglass(net, filterDim)
    outputs_2 = _hourglass(outputs_1[0], filterDim)
    model = tf.keras.Model(inputs=image, outputs=(outputs_1+outputs_2))
    return model

Define the loss function

In [7]:
def loss_fn(preds, labels):
    labels = tf.reshape(labels, [-1, imageHeight//4, imageWidth//4, 85])
    labels_offset_size = labels[...,80:]
    labels_offset_size_mask = tf.equal(labels_offset_size[...,0],1.0)
    labels_offset_size_obj = tf.boolean_mask(labels_offset_size, labels_offset_size_mask)
    #obj_num = tf.cast(labels_offset_size_obj.shape[0], tf.float32)
    obj_num = tf.cast(tf.shape(labels_offset_size_obj)[0], tf.float32)
    preds_offset = preds[1]
    preds_offset_obj = tf.boolean_mask(preds_offset, labels_offset_size_mask)
    loss_offset = tf.math.truediv(\
                      tf.reduce_sum(\
                          tf.math.abs(\
                              preds_offset_obj-labels_offset_size_obj[...,1:3])),
                      obj_num)
    preds_size = preds[2]
    preds_size_obj = tf.boolean_mask(preds_size, labels_offset_size_mask)
    loss_size = tf.math.truediv(\
                      tf.reduce_sum(\
                          tf.math.abs(\
                              preds_size_obj-labels_offset_size_obj[...,3:])),
                      obj_num)
    labels_k = labels[...,:80]
    labels_k_obj_mask = tf.equal(labels_k, 1.0)
    labels_k_noobj_mask = tf.less(labels_k, 1.0)
    labels_k_obj = tf.boolean_mask(labels_k, labels_k_obj_mask)
    labels_k_noobj = tf.boolean_mask(labels_k, labels_k_noobj_mask)
    keypoint = tf.nn.sigmoid(preds[0])
    preds_k_obj = tf.boolean_mask(keypoint, labels_k_obj_mask)
    preds_k_noobj = tf.boolean_mask(keypoint, labels_k_noobj_mask)
    loss_k_obj = tf.reduce_sum(\
                     tf.math.multiply(\
                         tf.math.pow((1.0-preds_k_obj), focal_alpha),\
                         tf.math.log(preds_k_obj+1e-5)))
    loss_k_noobj = tf.reduce_sum(\
                       tf.math.multiply(\
                           tf.math.multiply(\
                               tf.math.pow((1.0-labels_k_noobj), focal_beta),
                               tf.math.pow(preds_k_noobj, focal_alpha)),
                           tf.math.log(1.0-preds_k_noobj+1e-5)))
    loss_k = tf.truediv(\
                 tf.math.negative(loss_k_obj+loss_k_noobj),\
                 obj_num) 
    loss = loss_k + lambda_size*loss_size + lambda_offset*loss_offset
    return loss

Translate the output to prediction bounding boxes

In [ ]:
def predict(keypoint, offset, size):
    keypoint_nms = l.MaxPool2D(3, 1, 'same')(keypoint)
    keypoint_mask = tf.cast(tf.equal(keypoint, keypoint_nms), tf.float32)
    keypoint = keypoint * keypoint_mask
    keypoint_shape = tf.shape(keypoint)
    batch = keypoint_shape[0]
    height = keypoint_shape[1]
    width = keypoint_shape[2]
    category = keypoint_shape[3]
    keypoint_flat = tf.reshape(keypoint, [batch, -1])
    offset_flat = tf.reshape(offset, [batch, -1, 2])
    size_flat = tf.reshape(size, [batch, -1, 2])
    scores, indices = tf.math.top_k(keypoint_flat, k=10, sorted=True)
    classes = tf.cast(indices%category, tf.float32)
    indices = tf.cast(indices/category, tf.int32)
    x = tf.cast(indices%width, tf.float32)
    y = tf.cast(indices/height, tf.float32)
    offset_x = tf.batch_gather(offset_flat[...,1], indices)
    offset_y = tf.batch_gather(offset_flat[...,0], indices)
    size_w = tf.batch_gather(size_flat[...,1], indices)
    size_h = tf.batch_gather(size_flat[...,0], indices)
    x = x + offset_x
    y = y + offset_y
    xmin = (x - size_w/2)*4
    xmax = (x + size_w/2)*4
    ymin = (y - size_h/2)*4
    ymax = (y + size_h/2)*4
    bbox = tf.stack([xmin,xmax,ymin,ymax,scores,classes], axis=-1)
    return bbox

keypoint = tf.random.uniform([3,8,8,4])
offset = tf.random.uniform([3,8,8,2])
size = tf.random.uniform([3,8,8,2],minval=0,maxval=100)
bbox = predict(keypoint,offset,size)
bbox_mask = tf.greater(bbox[...,4],0.99)
bbox_nms = tf.boolean_mask(bbox, bbox_mask)

sess=tf.Session()
k,o,s,b,b_nms = sess.run([keypoint,offset,size, bbox, bbox_nms])
sess.close()

In [ ]:
a=tf.random.uniform([3,6,6,2])
b=tf.greater(a[...,0], 0.5)
c=tf.boolean_mask(a,b)
sess=tf.Session()
d,e,f=sess.run([a,b,c])
sess.close()

In [ ]:
tf.shape(a)

In [8]:
def predict(keypoint, offset, size):
    keypoint_nms = l.MaxPool2D(3, 1, 'same')(keypoint)
    keypoint_mask = tf.cast(tf.equal(keypoint, keypoint_nms), tf.float32)
    keypoint = keypoint * keypoint_mask
    keypoint_shape = tf.shape(keypoint)
    batch = keypoint_shape[0]
    height = keypoint_shape[1]
    width = keypoint_shape[2]
    category = keypoint_shape[3]
    keypoint_flat = tf.reshape(keypoint, [batch, -1])
    offset_flat = tf.reshape(offset, [batch, -1, 2])
    size_flat = tf.reshape(size, [batch, -1, 2])
    scores, indices = tf.math.top_k(keypoint_flat, k=10, sorted=True)
    classes = tf.cast(indices%category, tf.float32)
    indices = tf.cast(indices/category, tf.int32)
    x = tf.cast(indices%width, tf.float32)
    y = tf.cast(indices/height, tf.float32)
    offset_x = tf.batch_gather(offset_flat[...,1], indices)
    offset_y = tf.batch_gather(offset_flat[...,0], indices)
    size_w = tf.batch_gather(size_flat[...,1], indices)
    size_h = tf.batch_gather(size_flat[...,0], indices)
    x = x + offset_x
    y = y + offset_y
    xmin = (x - size_w/2)*4
    xmax = (x + size_w/2)*4
    ymin = (y - size_h/2)*4
    ymax = (y + size_h/2)*4
    bbox = tf.stack([xmin,xmax,ymin,ymax,scores,classes], axis=-1)
    return bbox
                                
def CenterModel(features, labels, mode, params):
    model = HourglassNetwork()
    training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    images = tf.reshape(features["images"], [-1, imageHeight, imageWidth, 3])
    #images = tf.reshape(features[0], [-1, imageHeight, imageWidth, 3])
    outputs = model(images, training)
    print('bbb')
    print(outputs)
    print(labels)
 
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "bboxes": predict(outputs[5], outputs[6], outputs[7])
    }
 
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, \
                                          export_outputs={'classify': tf.estimator.export.PredictOutput(predictions)})
 
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss_inter = loss_fn(outputs[1:4], labels)
    loss_final = loss_fn(outputs[5:], labels)
    loss = loss_inter + loss_final
 
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_global_step()
        '''
        boundaries = [5000, 60000, 80000]
        values = [0.1, 0.01, 0.001, 0.0001]
        learning_rate = tf.compat.v1.train.piecewise_constant(global_step, boundaries, values)
        tf.summary.scalar('learning_rate', learning_rate)
        optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
        '''
        optimizer = tf.train.AdamOptimizer()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(model.get_updates_for(features)):
            train_op = optimizer.minimize(loss=loss, global_step=global_step)
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    ''' 
    # Add evaluation metrics (for EVAL mode)
    m = tf.keras.metrics.sparse_top_k_categorical_accuracy(y_true=labels,  y_pred=logits)
    tf.summary.scalar('top-5_accuracy', m)
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    tf.summary.scalar('accuracy', accuracy[0]) 
    eval_metric_ops = {
        #"accuracy": tf.metrics.accuracy(labels=true_labels, predictions=predictions["classes"])}
        "accuracy": accuracy} 
        #"top-5 accuracy": (m.result(), m.update_state(y_true=labels, y_pred=logits))}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    '''

In [9]:
def main(_):
    my_feature_columns = []
    my_feature_columns.append(tf.feature_column.numeric_column(key='images', shape=(imageHeight,imageWidth, 3)))
    imagenet_classifier = tf.estimator.Estimator(model_fn=CenterModel, \
                                                 model_dir="/home/roy/AI/centermodel/", \
                                                 params={'feature_columns': my_feature_columns,})
    for _ in range(10):
        imagenet_classifier.train(input_fn=train_input_fn, steps=5000)
        #eval_results = imagenet_classifier.evaluate(input_fn=val_input_fn)
        #print(eval_results)
 
if __name__ == "__main__":
    tf.app.run(main)

I0924 14:32:48.736530 139942699554560 estimator.py:1790] Using default config.
I0924 14:32:48.738181 139942699554560 estimator.py:209] Using config: {'_model_dir': '/home/roy/AI/centermodel/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f46685f7da0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
W0924 

bbb
[<tf.Tensor 'model/re_lu_73/Relu:0' shape=(?, 128, 128, 256) dtype=float32>, <tf.Tensor 'model/tf_op_layer_Sigmoid/Sigmoid:0' shape=(?, 128, 128, 80) dtype=float32>, <tf.Tensor 'model/conv2d_85/BiasAdd:0' shape=(?, 128, 128, 2) dtype=float32>, <tf.Tensor 'model/conv2d_87/BiasAdd:0' shape=(?, 128, 128, 2) dtype=float32>, <tf.Tensor 'model/re_lu_147/Relu:0' shape=(?, 128, 128, 256) dtype=float32>, <tf.Tensor 'model/tf_op_layer_Sigmoid_1/Sigmoid_1:0' shape=(?, 128, 128, 80) dtype=float32>, <tf.Tensor 'model/conv2d_169/BiasAdd:0' shape=(?, 128, 128, 2) dtype=float32>, <tf.Tensor 'model/conv2d_171/BiasAdd:0' shape=(?, 128, 128, 2) dtype=float32>]
Tensor("IteratorGetNext:1", dtype=float32)


I0924 14:33:10.364601 139942699554560 estimator.py:1147] Done calling model_fn.
I0924 14:33:10.367045 139942699554560 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0924 14:33:19.488030 139942699554560 monitored_session.py:240] Graph was finalized.
W0924 14:33:19.793993 139942699554560 deprecation.py:323] From /home/roy/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
I0924 14:33:19.795884 139942699554560 saver.py:1280] Restoring parameters from /home/roy/AI/centermodel/model.ckpt-0
W0924 14:33:27.702707 139942699554560 deprecation.py:323] From /home/roy/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1066: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and wi

KeyboardInterrupt: 

In [ ]:
a=tf.math.log(0.001)
sess = tf.Session()
b=sess.run(a)
sess.close()

In [ ]:
b